# <center> Data leak analysis

In [16]:
%store -r item_cat
%store -r item
%store -r sub
%store -r shops
%store -r sales_test
%store -r sales_train
%store -r __ipy
%store -r __da

In [14]:
__ipy

Helper ipython script loaded


In [15]:
__da

Basic Data Analysis tools was loaded


### Train dataset

Some of values are negative like item_price and item_cnt_day. It can be a data leak

In [11]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.906234e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.726440e+03,2.612213e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


Sales_train has no null valeus

In [3]:
sales_train.isnull().apply(lambda x: x.value_counts())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
False,2935849,2935849,2935849,2935849,2935849,2935849


In [4]:
v = sales_train['date_block_num'].value_counts()
v[v.index.sort_values()].plot(kind='bar')

Duplicated items:

In [5]:
unique_items = sales_train.drop_duplicates(['date', 'item_id', 'shop_id'])
sales_train.shape[0] - unique_items.shape[0]

28

Some items duplicate each other. I will print items which items duplicate each other

In [6]:
#%%time
duplicated_cols = sales_train.groupby(['date', 'item_id', 'shop_id'])['date_block_num'].count().reset_index()
duplicated_cols = duplicated_cols[duplicated_cols['date_block_num'] > 1].rename(columns={'date_block_num': 'count'})
duplicated_cols['count'].value_counts()

2    28
Name: count, dtype: int64

In [7]:
duplicated_cols[['item_id', 'shop_id']].head()

,item_id,shop_id
79678,14050,31
89472,15702,38
94677,12133,25
116495,12133,16
161438,12133,25


### Test dataset

In [8]:
sales_train[['date_block_num', 'item_price', 'item_cnt_day']].describe()

,date_block_num,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,8.906234e+02,1.242641e+00
std,9.422988e+00,1.726440e+03,2.612213e+00
min,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.990000e+02,1.000000e+00
75%,2.300000e+01,9.990000e+02,1.000000e+00
max,3.300000e+01,3.079800e+05,2.169000e+03


Duplicated items:

In [9]:
unique_items = sales_test.drop_duplicates(['item_id', 'shop_id'])
sales_test.shape[0] - unique_items.shape[0]

0